In [1]:
import os
import json

from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

from dotenv import load_dotenv
load_dotenv()

streamer = os.environ.get("TWITCH_STREAMER")

if os.path.isfile(f"data/{streamer}.json"):
    with open(f"data/{streamer}.json") as f:
        json_data = json.load(f)
        first_id = json_data[0]["id"]
else:
    json_data = []
    first_id = None
print(streamer)
print(first_id)



crazzyccat
6b4be0ee-bba4-4f78-9bfd-42919587f142


In [2]:
transport = RequestsHTTPTransport(url="https://gql.twitch.tv/gql", headers={"client-id": os.environ.get("TWITCH_CLIENT_ID")})
client = Client(transport=transport)
query = gql(
"""
fragment UserPreview on User {
    id
    displayName
}

query GetChannelPredictions($login: String!, $after: Cursor, $first: Int!) {
    channel(name: $login) {
    id
    displayName
    url
    owner {
        profileImageURL(width: 150)
    }
    resolvedPredictionEvents(after: $after, first: $first) {
        edges {
        cursor
        node {
            title
            id
            createdAt
            endedAt
            lockedAt
            predictionWindowSeconds
            status
            createdBy {
            ...UserPreview
            }
            lockedBy {
            ...UserPreview
            }
            endedBy {
            ...UserPreview
            }
            outcomes {
            id
            title
            color
            totalPoints
            totalUsers
            topPredictors {
                points
                user {
                ...UserPreview
                }
            }
            }
            winningOutcome {
            id
            }
        }
        }
        pageInfo {
        hasNextPage
        hasPreviousPage
        }
    }
    }
}
"""
)

import time

def get_pages():
    query_data = []
    cursor = ""
    while True:
        time.sleep(1)
        params = {
            "login": streamer,
            "first": 25,
            "after": cursor
        }

        response = client.execute(query, variable_values=params)
        predictions = response["channel"]["resolvedPredictionEvents"]
        edges = predictions["edges"]
        pageinfo = predictions["pageInfo"]

        print(pageinfo)

        for edge in edges:
            cursor = edge["cursor"]

            prediction_id = edge["node"]["id"]
            if prediction_id == first_id:
                return query_data
            query_data.append(edge["node"])
        if pageinfo["hasNextPage"] == False:
            return query_data

query_data = get_pages()
data = query_data + json_data
print(len(data))



{'hasNextPage': True, 'hasPreviousPage': False}
{'hasNextPage': True, 'hasPreviousPage': False}
{'hasNextPage': True, 'hasPreviousPage': False}
{'hasNextPage': True, 'hasPreviousPage': False}
{'hasNextPage': True, 'hasPreviousPage': False}
{'hasNextPage': True, 'hasPreviousPage': False}
1866


In [3]:
with open(f"data/{streamer}.json", "w") as f:
    json.dump(data, f, indent=2, sort_keys=True, ensure_ascii=False)

